In [1]:
import os, glob
import numpy as np
import subprocess as s
import matplotlib.pyplot as plt
import warnings
import glob
warnings.filterwarnings("ignore")

from astropy.io import fits
from astropy.time import Time
from astropy.table import Table
from scipy.optimize import curve_fit


In [2]:
# Useful functions
def mkdir(path):
    # Create relevant directory
    if not os.path.exists(path):
        os.makedirs(path)

# Check and download data

In [3]:
def check_data(data_base, analysis_base, cluster, radius=60):
    os.chdir(data_base)
    
    xmm_browse = '/opt/xmm_download/./browse_extract.pl'
    
    f = open('check_data.txt', 'w')
    s.run([f'{xmm_browse}', 'table=xmmmaster', f'position={cluster}', 'name_resolver=ned', f'radius={radius/60.}',
           'format=Text'], stdout=f, stderr=f)
    f.close()
    
    os.chdir(analysis_base)

In [4]:
def download_data(obsid):    
    xmm_download = '/opt/xmm_download/./aioclient'
    
    f = open('download_data.txt', 'w')
    s.run([f'{xmm_download}', '-L', f'GET obsno={obsid} level=ODF'], stdout=f, stderr=f)
    f.close()

In [5]:
def unpack(obsid):    
    # Expand the main file
    fname = f'{obsid}.tar.gz'
    
    f = open('unpack_1.txt', 'w')
    s.run([f'tar', '-xvzf', fname], stdout=f, stderr=f)
    f.close()
    
    # Expand the observation file
    fname = fname = glob.glob('*.TAR')[0]
    
    f = open('unpack_2.txt', 'w')
    s.run([f'tar', '-xvzf', fname], stdout=f, stderr=f)
    f.close()

# Unpack data

# Set environment

In [6]:
def calib(data_path, analysis_path):
    os.environ['SAS_ODF'] = data_path
    f = open('cifbuild.txt', 'w')
    s.run(['cifbuild'], stdout=f, stderr=f)
    f.close()

    # Set cifbuild
    os.environ['SAS_CCF'] = analysis_path + 'ccf.cif'

    # Run odfingest
    f = open('odfingest.txt', 'w')
    s.run(['odfingest'], stdout=f, stderr=f)
    f.close()

    # Find SAS extension and set the environment
    os.environ['SAS_ODF'] = glob.glob(analysis_path + '*.SAS')[0]

In [7]:
def set_environ(analysis_path):
    # Set cifbuild
    os.environ['SAS_CCF'] = analysis_path + 'ccf.cif'
    
    # Find SAS extension and set the environment
    os.environ['SAS_ODF'] = glob.glob(analysis_path + '*.SAS')[0]

# Source analysis

# Reprocess dataset

In [8]:
def reprocess(instrument, analysis_path, OoT=False):
    # Create relevant directory
    mkdir(instrument)

    # Change to instrument directory
    os.chdir(instrument)

    # Keep log file in analysis directory
    if(OoT):
        f = open(analysis_path + '/' + instrument + '_OoT_reprocess.txt', 'w')
    else:
        f = open(analysis_path + '/' + instrument + '_reprocess.txt', 'w')
    
    # Check which instrument is requested and reprocess accordingly
    if(instrument == 'EMOS1' or instrument == 'EMOS2'):
        s.run(['emproc', 'selectinstruments=yes', instrument.lower()+'=yes'], stdout=f, stderr=f)
    
    elif(instrument == 'EPN'):
        if(OoT):
            #Create directory and change directory
            mkdir('OoT')
            os.chdir('OoT')
            
            # Reprocess
            s.run(['epproc', 'withoutoftime=yes'], stdout=f, stderr=f)
            
            # Rename the OoT file
            fname = glob.glob('*EPN*_ImagingEvts.ds')[0]
            index = fname.find('ImagingEvts.ds')
            new_fname = fname[:index] + 'OoT_' + fname[index:]
            
            s.run(['mv', fname, new_fname])
            
            # Change to instrument directory
            os.chdir('../')
        else:
            # Create directory and change directory
            mkdir('Main')
            os.chdir('Main')
            
            # Reprocess
            s.run(['epproc'], stdout=f, stderr=f)
            
            # Change to instrument directory
            os.chdir('../')
            
    # Handle exception appropriately
    else:
        os.chdir('../')
        os.unlink(f.name)
        s.run(['rm','-r', instrument])
        raise ValueError('Invalid instrument!!!')
                  
    f.close()
    os.chdir('../')

# Make lightcurves

In [9]:
def lc(instrument, analysis_path, bkg=False):
    # Change to instrument directory
    os.chdir(instrument)
    
    # Keep log file in analysis directory
    f = open(analysis_path + instrument + '_lc.txt', 'w')
    
    # Check which instrument is requested and process accordingly
    if(instrument == 'EPN'):
        os.chdir('Main')
        pattern = 4
    else:
        pattern = 12
        
    fname = glob.glob('*_ImagingEvts.ds')[0]

    s.run(['evselect', f'table={fname}', 'withrateset=Y', f'rateset=rate_{instrument}.fits', 
            'maketimecolumn=Y', 'timebinsize=100', 'makeratecolumn=Y', 
            f'expression=#XMMEA_{instrument[:2]} && (PI in [9000:12000]) && (PATTERN<={pattern})'], 
          stdout=f, stderr=f)
        
    if(instrument == 'EPN'):
        s.run(['cp', f'rate_{instrument}.fits', f'../OoT/rate_{instrument}.fits'])

    
    f.close()
    os.chdir(analysis_path)

# Fit and plot gaussian

In [10]:
def plotting(x, y, y_err, model, xlim, ObsID, instrument, figures_path, xscale='linear', bkg=False):
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8,8), sharex=True, gridspec_kw={'height_ratios': [3, 1]})
    ax1.tick_params(direction='in', which='both', length=5, labelsize=15)
    ax2.tick_params(direction='in', which='both', length=5, labelsize=15)

    plt.suptitle('RATE')
    
    ax1.set_ylabel(r'N', fontsize = 15.0)
    ax1.set_xscale(xscale)
    
    ax1.set_xlim(0, xlim[1])

    ax1.errorbar(x, y, yerr=y_err, fmt='o', capsize=2, label='Data')
    ax1.plot(x, model, label='Fit')
    ax1.legend(loc='best', fontsize=15.0)

    ax2.set_ylabel(r'Ratio', fontsize = 15.0)
    ax2.set_ylim(0.5,1.5)
    ax2.axhline(y=1.)

    ax2.errorbar(x, y/model, yerr=y_err/model, fmt='o', capsize=2)

    ax2.set_xlabel('Rate [cts/s]', fontsize = 15.0)

    plt.subplots_adjust(hspace=0.1)
    
    if not bkg:
        plt.savefig(figures_path+f'{ObsID}_{instrument}_rate_fit.png', bbox_inches='tight')
    else:
        plt.savefig(figures_path+f'{instrument}_bkg_rate_fit.png', bbox_inches='tight')
    
    plt.show()

In [11]:
def fit(instrument, analysis_path, ObsID, bkg=False):
    # Change to instrument directory
    if(instrument == 'EPN'):
        os.chdir(instrument + '/Main/')
    else:
        os.chdir(instrument)
        
    # Load file and data
    if not bkg:
        fname = 'rate_' + instrument + '.fits'
    else:
        fname = 'rate_' + instrument + '_bkg.fits'
        
    hdul = fits.open(fname)
    data = hdul[1].data

    time = data['time']
    rate = data['rate']
    rate_err = data['error']

    hdul.close()
    os.chdir(analysis_path)
    
    # Delete zero rates
    where = np.where(rate == 0)[0]

    time = np.delete(time, where)
    rate = np.delete(rate, where)
    rate_err = np.delete(rate_err, where)
    
    # Make histogram
    hist, bin_edges = np.histogram(np.log10(rate), bins=20)
        
    bin_mid = 0.5*(bin_edges[1:] + bin_edges[:-1])
    bin_mid = 10**(bin_mid)
    hist_err = np.sqrt(hist)
    
    # Delete zero counts in bins
    where = np.where(hist_err == 0)[0]

    bin_mid = np.delete(bin_mid, where)
    hist = np.delete(hist, where)
    hist_err = np.delete(hist_err, where)
    
    # Define gaussian function
    gaussian = lambda x, amp, cen, sig: amp * np.exp(-(x-cen)**2 / sig**2)

    # Fit
    init_vals = [max(hist), np.mean(bin_mid), np.std(bin_mid)]
    
    best_vals, covar = curve_fit(gaussian, bin_mid, hist, p0=init_vals, sigma=1/hist_err, bounds=((0, 0, 0),
                                                                                        (np.inf, np.inf, np.inf)),
                                maxfev=100000)
    # Plot
    if not bkg:
        plotting(bin_mid, hist, hist_err, gaussian(bin_mid, *best_vals), 
                 [best_vals[1]-2*best_vals[2], best_vals[1]+2*best_vals[2]], ObsID, 
                 instrument, figures_path)
    else:
        plotting(bin_mid, hist, hist_err, gaussian(bin_mid, *best_vals), 
                 [best_vals[1]-2*best_vals[2], best_vals[1]+2*best_vals[2]], ObsID, 
                 instrument, figures_path, bkg=True)
    
    return best_vals[1], best_vals[2]

# Highly flared data

In [12]:
def flared_fit(instrument, analysis_path, ObsID, EMOS_cts_limit = 0.6, bkg=False):
    # Change to instrument directory
    if(instrument == 'EPN'):
        os.chdir(instrument + '/Main/')
    else:
        os.chdir(instrument)
        
    # Load file and data
    if not bkg:
        fname = 'rate_' + instrument + '.fits'
    else:
        fname = 'rate_' + instrument + '_bkg.fits'
        
    hdul = fits.open(fname)
    data = hdul[1].data

    time = data['time']
    rate = data['rate']
    rate_err = data['error']

    hdul.close()
    os.chdir(analysis_path)
    
    # Delete zero rates
    where = np.where(rate == 0)[0]

    time = np.delete(time, where)
    rate = np.delete(rate, where)
    rate_err = np.delete(rate_err, where)
    
    # Make histogram
    hist, bin_edges = np.histogram(np.log10(rate), bins=50)
        
    bin_mid = 0.5*(bin_edges[1:] + bin_edges[:-1])
    bin_mid = 10**(bin_mid)
    hist_err = np.sqrt(hist)
    
    # Delete zero counts in bins
    where = np.where(hist_err == 0)[0]

    bin_mid = np.delete(bin_mid, where)
    hist = np.delete(hist, where)
    hist_err = np.delete(hist_err, where)
    
    # Define gaussian function
    gaussian = lambda x, amp, cen, sig: amp * np.exp(-(x-cen)**2 / sig**2)

    # Fit
    best_vals = [max(hist), np.mean(bin_mid), np.std(bin_mid)]
    
    for i in range(2):
        best_vals, covar = curve_fit(gaussian, bin_mid, hist, p0=best_vals, sigma=1/hist_err, bounds=((0, 0, 0),
                                                                                        (np.inf, np.inf, np.inf)),
                                maxfev=10000)
        
        if(instrument == 'EPN'):
            hist = hist[bin_mid < 1.3]
            hist_err = hist_err[bin_mid < 1.3]
            bin_mid = bin_mid[bin_mid < 1.3]
            
        else:
            hist = hist[bin_mid < EMOS_cts_limit]
            hist_err = hist_err[bin_mid < EMOS_cts_limit]
            bin_mid = bin_mid[bin_mid < EMOS_cts_limit]
        
    # Plot
    if not bkg:
        plotting(bin_mid, hist, hist_err, gaussian(bin_mid, *best_vals), 
                 [best_vals[1]-2*best_vals[2], best_vals[1]+2*best_vals[2]], ObsID, 
                 instrument, figures_path)
    else:
        plotting(bin_mid, hist, hist_err, gaussian(bin_mid, *best_vals), 
                 [best_vals[1]-2*best_vals[2], best_vals[1]+2*best_vals[2]], ObsID, 
                 instrument, figures_path, bkg=True)
    
    return best_vals[1], best_vals[2]

# Make GTI and filter

In [13]:
def gti_filter(instrument, analysis_path, mean, sigma, OoT=False):
    # Keep log file 
    f = open(analysis_path + instrument + '_filter.txt', 'w')
    
    if(instrument == 'EPN'):
        if(OoT):
            os.chdir(instrument + '/OoT/')
            os.rename(f.name, analysis_path + instrument + '_OoT_filter.txt')
        else:
            os.chdir(instrument + '/Main/')
            os.rename(f.name, analysis_path + instrument + '_main_filter.txt')
    else:
        os.chdir(instrument)
        
    # Create GTI and filter
    fname = 'rate_' + instrument + '.fits'
    
    s.run(['tabgtigen', f'table={fname}', f'expression=RATE<={mean+2*sigma}', f'gtiset={instrument}_gti.fits'], 
          stdout=f, stderr=f)

    fname = glob.glob('*_ImagingEvts.ds')[0]
    
    s.run(['evselect', f'table={fname}', 'withfilteredset=Y', f'filteredset={instrument}_clean.fits', 
            'destruct=Y', 'keepfilteroutput=T', 
            f'expression=#XMMEA_{instrument} && gti({instrument}_gti.fits,TIME) && (PI>150)'], stdout=f,stderr=f)
         
    f.close()
    os.chdir(analysis_path)

# Create Images

In [14]:
def image(instrument, analysis_path, OoT=False):
    # Keep log file 
    f = open(analysis_path + instrument + '_image.txt', 'w')
    
    if(instrument == 'EPN'):
        pattern = 4
        
        if(OoT):
            os.chdir(instrument + '/OoT/')
            os.rename(f.name, analysis_path + instrument + '_OoT_image.txt')
        else:
            os.chdir(instrument + '/Main/')
            os.rename(f.name, analysis_path + instrument + '_main_image.txt')
    else:
        pattern = 12
        os.chdir(instrument)
        
    # Create image
    fname = instrument + '_clean.fits'
    
    s.run(['evselect', f'table={fname}', 'xcolumn=X', 'ycolumn=Y', 'imagebinning=binSize', 'ximagebinsize=80',
            'yimagebinsize=80', 'withimageset=true', f'imageset={instrument}_counts_nr.img', 
            f'expression=#XMMEA_{instrument[:2]}&&(PATTERN<={pattern})&&(PI>=500)&&(PI<=7000)'], stdout=f, stderr=f)
        
    # Create exposure map
    fname = glob.glob('*_AttHk.ds')[0]
    
    s.run(['eexpmap', f'imageset={instrument}_counts_nr.img', f'attitudeset={fname}', 
           f'eventset={instrument}_clean.fits', f'expimageset={instrument}_exp_nr.img', 'pimin=500', 'pimax=7000'],
           stdout=f, stderr=f)
    
    f.close()
    
    os.chdir(analysis_path)

# OoT image correction

In [15]:
def oot_corr(frame, analysis_path):
    # Keep log file 
    f = open(analysis_path + 'EPN_oot_image_corr.txt', 'w')
    os.chdir(analysis_path + 'EPN/OoT/')
    
    # Scale OoT image
    if(frame == 'eff'):
        s.run(['farith', 'EPN_counts.img', '0.023', 'EPN_rescaled_counts.img', 'MUL', 'clobber=yes'], 
              stdout=f, stderr=f)
        
    elif(frame == 'ff'):
        s.run(['farith', 'EPN_counts.img', '0.063', 'EPN_rescaled_counts.img', 'MUL', 'clobber=yes'], 
              stdout=f, stderr=f)
        
    # Subtract from PN image
    os.chdir(analysis_path + 'EPN/main/')
    
    fname = analysis_path + 'EPN/OoT/EPN_rescaled_counts.img'
    s.run(['farith', 'EPN_counts.img', f'{fname}', 'EPN_corr_counts.img', 'SUB', 'clobber=yes'], 
          stdout=f, stderr=f)
    
    f.close()
    os.chdir(analysis_path)

# Background analysis

# Reproject background files

In [16]:
def bkg_reprojection(cluster, instrument, inst_filter, analysis_path):
    # Keep log file 
    f = open(analysis_path + instrument + '_bkg_reproject.txt', 'w')
    
    if(instrument == 'EPN'):
        os.chdir(instrument + '/Main/')
        os.rename(f.name, analysis_path + instrument + '_bkg_main_reproject.txt')
    else:
        os.chdir(instrument)
        
    # Read in key headers
    fname = analysis_path + f'EMOS1/EMOS1_clean.fits'
    hdul = fits.open(fname)
    data = hdul[0].header

    ra = data['ra_pnt']
    dec = data['dec_pnt']
    pa = data['pa_pnt']

    hdul.close()
    
    # Locate the bkg file first
    bkg_path = f'/Users/anweshmajumder/Desktop/Grad/My_papers/Paper-2/{cluster}/Data/'
    
    if((instrument == 'EPN') and (frame == 'eff')):
        if(inst_filter == 't'):
            fname = glob.glob(bkg_path + instrument + 'tef*')[0]
    
        elif(inst_filter == 'm'):
            fname = glob.glob(bkg_path + instrument + 'mef*')[0]
            
    elif((instrument == 'EPN') and (frame == 'ff')):
        if(inst_filter == 't'):
            fname = glob.glob(bkg_path + instrument + 'tff*')[0]
    
        elif(inst_filter == 'm'):
            fname = glob.glob(bkg_path + instrument + 'mff*')[0]
            
    else:
        if(inst_filter == 't'):
            fname = glob.glob(bkg_path + instrument + 't*')[0]
    
        elif(inst_filter == 'm'):
            fname = glob.glob(bkg_path + instrument + 'm*')[0]
        
    # Copy the necessary background file
    s.run(['cp', f'{fname}', f'{instrument}_bkg_evt.fits'], stdout=f,stderr=f)
        
    # Then reproject
    s.run(['evproject', f'eventset={instrument}_bkg_evt.fits', 'attsource=fixed', f'attra={ra}', f'attdec={dec}', 
           f'attapos={pa}'], stdout=f,stderr=f)
    
    f.close()
    
    os.chdir(analysis_path)

In [17]:
def bkg_lc(instrument, analysis_path):
    os.chdir(instrument)
    
    # Keep log file in analysis directory
    f = open(analysis_path + instrument + '_bkg_lc.txt', 'w')
    
    # Check which instrument is requested and process accordingly
    if(instrument == 'EPN'):
        os.chdir('Main')
        pattern = 4
    else:
        pattern = 12
    
    fname = instrument + '_bkg_evt.fits'

    s.run(['evselect', f'table={fname}', 'withrateset=Y', f'rateset=rate_{instrument}_bkg.fits', 
            'maketimecolumn=Y', 'timebinsize=100', 'makeratecolumn=Y', 
            f'expression=#XMMEA_{instrument[:2]} && (PI in [9000:12000]) && (PATTERN<={pattern})'], 
          stdout=f, stderr=f)
    
    f.close()
    
    os.chdir(analysis_path)

In [18]:
def bkg_gti_filter(instrument, analysis_path, mean, sigma):
    # Keep log file 
    f = open(analysis_path + instrument + '_bkg_filter.txt', 'w')
    
    if(instrument == 'EPN'):
        os.chdir(instrument + '/Main/')
        os.rename(f.name, analysis_path + instrument + '_bkg_filter.txt')
    else:
        os.chdir(instrument)
        
    # Create GTI and filter
    fname = 'rate_' + instrument + '_bkg.fits'
    
    s.run(['tabgtigen', f'table={fname}', f'expression=RATE<={mean+2*sigma}', f'gtiset={instrument}_bkg_gti.fits'], 
          stdout=f, stderr=f)
    
    fname = instrument + '_bkg_evt.fits'
    
    s.run(['evselect', f'table={fname}', 'withfilteredset=Y', f'filteredset={instrument}_bkg_clean.fits', 
            'destruct=Y', 'keepfilteroutput=T', 
            f'expression=#XMMEA_{instrument[:2]} && gti({instrument}_bkg_gti.fits,TIME) && (PI>150)'], 
            stdout=f,stderr=f)
    
    # Create OofFOV file as well
    s.run(['evselect', f'table={fname}', 'withfilteredset=Y', f'filteredset={instrument}_bkg_oofov.fits', 
            'destruct=Y', 'keepfilteroutput=T', 
            f'expression=#XMMEA_16 && gti({instrument}_bkg_gti.fits,TIME) && (PI>150)'], 
            stdout=f,stderr=f)
         
    f.close()
    os.chdir(analysis_path)

# Calculate scaling factor for background

In [19]:
def bkg_scaling(cluster, instrument, analysis_path, after_2005 = False):
    # Keep log file 
    f = open(analysis_path + instrument + '_bkg_scaling.txt', 'w')
    
    if(instrument == 'EPN'):
        pattern = 4
        
        os.chdir(instrument + '/Main/')
        os.rename(f.name, analysis_path + instrument + '_bkg_main_scaling.txt')
    else:
        pattern = 12
        
        os.chdir(instrument)
        
    #-------------------------------- BACKGROUND ANALYSIS -----------------------------------
    bkg_path = f'/Users/anweshmajumder/Desktop/Grad/My_papers/Paper-2/{cluster}/Data/'
    region_file = bkg_path + instrument + 'fwc.cxc'
    
    fname = f'{instrument}_bkg_oofov.fits'
    
    # Account for loss of EMOS1 CCDs afer and before 2005
    if(instrument == 'EMOS1'):
        if(after_2005):
            s.run(['evselect', f'table={fname}', 'withfilteredset=Y', f'filteredset={instrument}_bkgtime_oofov.fits', 
                    'destruct=Y', 'keepfilteroutput=T', 
                    f'expression=#XMMEA_16&&(PATTERN<={pattern})&&(PI in [9000:12000])\
                    &&(CCDNR.ne.6)\
                    &&region({region_file})'], stdout=f, stderr=f)
            
            s.run(['evselect', f'table={instrument}_bkgtime_oofov.fits', 'xcolumn=X', 'ycolumn=Y', 
                   'imagebinning=binSize', 'ximagebinsize=20', 'yimagebinsize=20', 'withimageset=true', 
                   f'imageset={instrument}_oofov_bkg_counts.img'], stdout=f, stderr=f)
            
        else:
            # Constraining the bkg events uptil 2005
            s.run(['evselect', f'table={fname}', 'withrateset=Y', f'rateset=rate_{instrument}_bkgclean.fits', 
                    'maketimecolumn=Y', 'timebinsize=100', 'makeratecolumn=Y', 
                    f'expression=#XMMEA_16&&(PI in [9000:12000])&&(PATTERN<={pattern})'], 
                      stdout=f, stderr=f)
            
            s.run(['tabgtigen', f'table=rate_{instrument}_bkgclean.fits', f'expression=TIME<226713600', 
                   f'gtiset={instrument}_bkgtime_gti.fits'], stdout=f, stderr=f)
            
            
            # Then filtering
            s.run(['evselect', f'table={fname}', 'withfilteredset=Y', f'filteredset={instrument}_bkgtime_oofov.fits', 
                    'destruct=Y', 'keepfilteroutput=T', 
                    f'expression=#XMMEA_16&&(PATTERN<={pattern})&&(PI in [9000:12000])&&\
                    gti({instrument}_bkgtime_gti.fits,TIME)&&region({region_file})'], stdout=f, stderr=f)
            
            s.run(['evselect', f'table={instrument}_bkgtime_oofov.fits', 'xcolumn=X', 'ycolumn=Y', 
                   'imagebinning=binSize', 'ximagebinsize=20', 'yimagebinsize=20', 'withimageset=true', 
                   f'imageset={instrument}_oofov_bkg_counts.img'], stdout=f, stderr=f)
    
    # Others get processed normally
    elif(instrument == 'EMOS2'):
        s.run(['evselect', f'table={fname}', 'withfilteredset=Y', f'filteredset={instrument}_bkgtime_oofov.fits', 
                'destruct=Y', 'keepfilteroutput=T', 
                f'expression=#XMMEA_16&&(PATTERN<={pattern})&&(PI in [9000:12000])&&region({region_file})'], 
                stdout=f, stderr=f)
            
        s.run(['evselect', f'table={instrument}_bkgtime_oofov.fits', 'xcolumn=X', 'ycolumn=Y', 
                'imagebinning=binSize', 'ximagebinsize=20', 'yimagebinsize=20', 'withimageset=true', 
                f'imageset={instrument}_oofov_bkg_counts.img'], stdout=f, stderr=f)
    
    else:
        fname = f'{instrument}_bkg_clean.fits'
        
        s.run(['evselect', f'table={fname}', 'withfilteredset=Y', f'filteredset={instrument}_bkgtime_oofov.fits', 
                'destruct=Y', 'keepfilteroutput=T', 
                f'expression=#XMMEA_EP&&(PATTERN<={pattern})&&(PI in [9000:12000])'], 
                stdout=f, stderr=f)
            
        s.run(['evselect', f'table={instrument}_bkgtime_oofov.fits', 'xcolumn=X', 'ycolumn=Y', 
                'imagebinning=binSize', 'ximagebinsize=20', 'yimagebinsize=20', 'withimageset=true', 
                f'imageset={instrument}_oofov_bkg_counts.img'], stdout=f, stderr=f)
        
        
        
    # Calculate background counts
    fname = instrument + '_oofov_bkg_counts.img'
    hdul = fits.open(fname)
    data = hdul[0].data

    bkg_counts = np.sum(data)
    hdul.close()
    
    # Read livetime for bkg
    fname = f'{instrument}_bkgtime_oofov.fits'
    hdul = fits.open(fname)
    data = hdul[1].header

    bkg_livetime = data['LIVETI02']
    
    hdul.close()
    
    #-------------------------------- OBSERVATION ANALYSIS ---------------------------------------------------
    fname = glob.glob('*_ImagingEvts.ds')[0]
    
    # Create an oofov event file as well
    bkg_path = f'/Users/anweshmajumder/Desktop/Grad/My_papers/Paper-2/{cluster}/Data/'
    region_file = bkg_path + instrument + 'fwc.cxc'
        
    if(instrument == 'EPN'):
        s.run(['evselect', f'table={fname}', 'withfilteredset=Y', f'filteredset={instrument}_oofov_clean.fits', 
            'destruct=Y', 'keepfilteroutput=T', 
            f'expression=#XMMEA_16&&(PATTERN<={pattern})&&(PI in [9000:12000])&&gti({instrument}_gti.fits,TIME)\
            &&(PI>150)'], stdout=f,stderr=f)
    
        s.run(['evselect', f'table={instrument}_oofov_clean.fits', 'xcolumn=X', 'ycolumn=Y', 'imagebinning=binSize', 
               'ximagebinsize=20', 'yimagebinsize=20', 'withimageset=true', f'imageset={instrument}_oofov_counts.img'],
               stdout=f, stderr=f)
    
    else:
        s.run(['evselect', f'table={fname}', 'withfilteredset=Y', f'filteredset={instrument}_oofov_clean.fits', 
                'destruct=Y', 'keepfilteroutput=T', 
                f'expression=#XMMEA_16&&(PATTERN<={pattern})&&(PI in [9000:12000])&&gti({instrument}_gti.fits,TIME)&&\
                region({region_file})&&(PI>150)'], stdout=f,stderr=f)
    
        s.run(['evselect', f'table={instrument}_oofov_clean.fits', 'xcolumn=X', 'ycolumn=Y', 'imagebinning=binSize', 
               'ximagebinsize=20', 'yimagebinsize=20', 'withimageset=true', f'imageset={instrument}_oofov_counts.img'],
               stdout=f, stderr=f)
        
    # Read the counts
    fname = instrument + '_oofov_counts.img'
    hdul = fits.open(fname)
    data = hdul[0].data

    obs_counts = np.sum(data)
    
    hdul.close()
    
    # Read livetime for source
    fname = instrument + '_oofov_clean.fits'
    hdul = fits.open(fname)
    data = hdul[1].header

    obs_livetime = data['LIVETI02']
    
    hdul.close()
    
    f.close()
    os.chdir(analysis_path)
    
    #print( (bkg_counts/bkg_livetime) / (obs_counts/obs_livetime) )
    return bkg_counts / obs_counts

# Image production background files, scaling, subtraction

In [20]:
def bkg_image(instrument, analysis_path, scaling, after_2005=False):
    # Keep log file 
    f = open(analysis_path + instrument + '_bkg_image.txt', 'w')
    
    if(instrument == 'EPN'):
        pattern = 4
        imgname = 'EPN_corr_counts_nr.img'
        
        os.chdir(instrument + '/Main/')
        os.rename(f.name, analysis_path + instrument + '_bkg_image.txt')
    else:
        pattern = 12
        imgname = f'{instrument}_counts.img'
        
        os.chdir(instrument)
        
    # Create image
    fname = instrument + '_bkg_clean.fits'
    
    if((instrument == 'EMOS1') and (after_2005 == True)):
        s.run(['evselect', f'table={fname}', 'xcolumn=X', 'ycolumn=Y', 'imagebinning=binSize', 'ximagebinsize=20',
                'yimagebinsize=20', 'withimageset=true', f'imageset={instrument}_bkg_counts.img', 
                f'expression=(#XMMEA_{instrument[:2]})&&(PATTERN<={pattern})&&(PI in [500:7000])\
                &&(CCDNR.ne.6)'], stdout=f, stderr=f)
        
    else:
        s.run(['evselect', f'table={fname}', 'xcolumn=X', 'ycolumn=Y', 'imagebinning=binSize', 'ximagebinsize=20',
                'yimagebinsize=20', 'withimageset=true', f'imageset={instrument}_bkg_counts.img', 
                f'expression=#XMMEA_{instrument[:2]}&&(PATTERN<={pattern})&&(PI in [500:7000])'], 
              stdout=f, stderr=f)
          
    # Add header
    s.run(['fparkey', f'{1/scaling}', f'{instrument}_bkg_counts.img[0]', f'bkgnorm', 'add=yes'], stdout=f, stderr=f)
    
    # Scaling
    #s.run(['farith', f'{instrument}_bkg_counts.img', f'{scaling}', f'{instrument}_rescaled_bkg_counts.img', 
    #       'DIV', 'clobber=yes'], stdout=f, stderr=f)
    
    # Subtraction
    #s.run(['farith', f'{imgname}', f'{instrument}_rescaled_bkg_counts.img', f'{instrument}_bkgsub_counts.img', 
    #       'SUB', 'null=yes', 'blank=0', 'clobber=yes'], stdout=f, stderr=f)
    
    # Exposure correction
    #s.run(['farith', f'{instrument}_bkgsub_counts.img', f'{instrument}_exp.img', f'{instrument}_flux.img', 
    #       'DIV', 'clobber=yes'], stdout=f, stderr=f)
    
    f.close()
    
    os.chdir(analysis_path)

# Add images

In [31]:
def add_image(cluster, img_paths, exp_paths, destination, instrument):
    # Keep log file 
    f = open(destination + instrument + '_add_image.txt', 'w')
    
    if (len(img_paths) is not len(exp_paths)):
        raise ValueError("Number of images and exposures must be same")
    
    images = " ".join(img_paths)
    exposures = " ".join(exp_paths)
    
    s.run(['emosaic', f'imagesets={images}', f'mosaicedset={destination}{instrument}_{cluster}_exp_nr.img', 
           'withexposure=no'], stdout=f, stderr=f)
    
    #s.run(['emosaic', f'imagesets={images}', f'mosaicedset={destination}{instrument}_{cluster}_exp_corr_nr.img', 
    #       'withexposure=yes', f'exposuresets={exposures}'], stdout=f, stderr=f)
    
    f.close()

In [23]:
cluster = 'MACS J2135.2-0102'
instruments = ['EMOS1', 'EMOS2']

figures_path = f'/Users/anweshmajumder/Desktop/Grad/My_papers/Paper-2/{cluster.replace(" ", "_")}/figures/'

# Make appropriate directories
data_base = f'/Users/anweshmajumder/Desktop/Grad/My_papers/Paper-2/{cluster.replace(" ", "_")}/Data/XMM_obs/'
analysis_base = f'/Users/anweshmajumder/Desktop/Grad/My_papers/Paper-2/{cluster.replace(" ", "_")}/Analysis/'

mkdir(data_base)
mkdir(analysis_base)
mkdir(figures_path)

# Check XMM data
check_data(data_base, analysis_base, cluster, radius=1000)

# Read obsids
ObsIDs = np.loadtxt(data_base + 'check_data.txt', dtype=str, skiprows=5, delimiter='|', usecols=(1))

In [222]:
ObsIDs = ObsIDs[:-1]
ObsIDs

array(['0021540501', '0021540101', '0723800201', '0723800101'],
      dtype='<U10')

In [266]:
#ObsIDs = np.delete(ObsIDs,-2)
ObsID = ObsIDs

In [24]:
for i, ObsID in enumerate(ObsIDs):
    data_path = f'/Users/anweshmajumder/Desktop/Grad/My_papers/Paper-2/{cluster.replace(" ", "_")}/Data/XMM_obs/{ObsID}/'
    analysis_path = f'/Users/anweshmajumder/Desktop/Grad/My_papers/Paper-2/{cluster.replace(" ", "_")}/Analysis/{ObsID}/'
    
    #mkdir(data_path)
    #mkdir(analysis_path)
    
    # Change to data directory
    #os.chdir(data_path)
    
    # Download the data
    #download_data(ObsID)
    
    # Unpack data
    #unpack(ObsID)

    # Change to analysis directory
    os.chdir(analysis_path)
    
    # Set the environment
    #calib(data_path, analysis_path)
    set_environ(analysis_path)

    # Reprocess instruments
    #for instrument in instruments:
    #    reprocess(instrument, analysis_path)

    # Create light curves
    #for instrument in instruments:
    #    lc(instrument, analysis_path)

    # Fit gaussian, plot and filter
    #for instrument in instruments:
    #    mean, sigma = fit(instrument, analysis_path, ObsID)
    #    mean, sigma = flared_fit(instrument, analysis_path, ObsID, 0.5)
    #    print(f'ObsID: {ObsID}; instrument: {instrument}; Mean =  {mean}; sigma = {sigma}')
    #    gti_filter(instrument, analysis_path, mean, sigma)
        
        #break
    
    # Create images
    for instrument in instruments:
        image(instrument, analysis_path)
    
    # Reproject background files
    #for instrument in instruments:
    #    bkg_reprojection(cluster, instrument, inst_filter[i], analysis_path)
    
    # Filter background files
    # Create light curves
    #for instrument in instruments:
    #    bkg_lc(instrument, analysis_path)
        #break

    # Fit gaussian, plot and filter
    #for instrument in instruments:
    #    mean, sigma = fit(instrument, analysis_path, ObsID, bkg=True)
    #    print(f'Mean =  {mean}; sigma = {sigma}')
    #    bkg_gti_filter(instrument, analysis_path, mean, sigma)
        
        #break
    
    # Calculate scaling factor
    #for instrument in instruments:
    #    scaling = bkg_scaling(cluster, instrument, analysis_path, after_2005s[i])
    #    print('Bkg cts/Src cts [9-12 keV OoFOV]: ', scaling)
    #    bkg_image(instrument, analysis_path, scaling, after_2005s[i])
        #break
    
    #break

In [263]:
ObsID

array('0146510301', dtype='<U10')

In [32]:
# Add images
added_path = f'/Users/anweshmajumder/Desktop/Grad/My_papers/Paper-2/{cluster.replace(" ", "_")}/Analysis/'
img_path = []
exp_path = []

for ObsID in ObsIDs:
    img_path.append(f'/Users/anweshmajumder/Desktop/Grad/My_papers/Paper-2/{cluster.replace(" ", "_")}/Analysis/{ObsID}/EMOS1/EMOS1_counts_nr.img')
    img_path.append(f'/Users/anweshmajumder/Desktop/Grad/My_papers/Paper-2/{cluster.replace(" ", "_")}/Analysis/{ObsID}/EMOS2/EMOS2_counts_nr.img')
                    
    exp_path.append(f'/Users/anweshmajumder/Desktop/Grad/My_papers/Paper-2/{cluster.replace(" ", "_")}/Analysis/{ObsID}/EMOS1/EMOS1_exp_nr.img')
    exp_path.append(f'/Users/anweshmajumder/Desktop/Grad/My_papers/Paper-2/{cluster.replace(" ", "_")}/Analysis/{ObsID}/EMOS2/EMOS2_exp_nr.img')
        
#add_image(cluster, img_path, exp_path, added_path, 'EMOS')
add_image(cluster, exp_path, exp_path, added_path, 'EMOS')

In [125]:
exp_path

['/Users/anweshmajumder/Desktop/Grad/My_papers/Paper-2/Cygnus_A/Analysis/0302800201/EMOS1/EMOS1_exp_nr.img',
 '/Users/anweshmajumder/Desktop/Grad/My_papers/Paper-2/Cygnus_A/Analysis/0302800201/EMOS2/EMOS2_exp_nr.img',
 '/Users/anweshmajumder/Desktop/Grad/My_papers/Paper-2/Cygnus_A/Analysis/0302800101/EMOS1/EMOS1_exp_nr.img',
 '/Users/anweshmajumder/Desktop/Grad/My_papers/Paper-2/Cygnus_A/Analysis/0302800101/EMOS2/EMOS2_exp_nr.img']

# Add bkgnorm parameter

In [262]:
#scaling = np.array([])
exposure = np.array([])
exposure_source = np.array([])

for i, ObsID in enumerate(ObsIDs):
    data_path = f'/Users/anweshmajumder/Desktop/Grad/My_papers/Paper-2/{cluster}/Data/XMM_obs/{ObsID}/'
    analysis_path = f'/Users/anweshmajumder/Desktop/Grad/My_papers/Paper-2/{cluster}/Analysis/{ObsID}/'

    # Change to analysis directory
    for instrument in instruments:
        os.chdir(analysis_path+'/'+instrument)
        
        #fname = instrument + '_bkg_counts.img'
        #hdul = fits.open(fname)
        #data = hdul[0].header

        #scaling = np.append(scaling, data['bkgnorm'])
        #exposure = np.append(exposure, data['exposure'])
    
        #hdul.close()
        
        fname = instrument + '_counts.img'
        hdul = fits.open(fname)
        data = hdul[0].header

        exposure_source = np.append(exposure_source, data['exposure'])
    
        hdul.close()
        
#weighted_norm = np.average(scaling, weights=exposure_source)
#bkg_exposure = np.sum(exposure)
src_exposure = np.sum(exposure_source)

os.chdir(f'/Users/anweshmajumder/Desktop/Grad/My_papers/Paper-2/{cluster}/Analysis/')
s.run(['fparkey', f'{1.}', f'EMOS_NGC5846_bkg_counts.img[0]', f'bkgnorm', 'add=yes'])
s.run(['fparkey', f'{src_exposure}', f'EMOS_NGC5846_bkg_counts.img[0]', f'exposure', 'add=yes'])
s.run(['fparkey', f'{src_exposure}', f'EMOS_NGC5846_counts.img[0]', f'exposure', 'add=yes'])

TypeError: iteration over a 0-d array

# Unsharp masking

In [19]:
from scipy.ndimage import gaussian_filter

In [29]:
cluster = 'Phoenix'
added_path = f'/Users/anweshmajumder/Desktop/Grad/My_papers/Paper-2/{cluster}/Analysis/'

fname = added_path + f'EMOS_Phoenix_wops_exp_corr.img'
hdul = fits.open(fname)
data = hdul[0].data
header = hdul[0].header
    
hdul.close()

smooth_1_data = gaussian_filter(data, sigma=1)
smooth_10_data = gaussian_filter(data, sigma=4)

unsharp_data = smooth_1_data - smooth_10_data

hdu = fits.PrimaryHDU(unsharp_data, header)
hdu.writeto(added_path + f'EMOS_{cluster}_unsharp_1.img', overwrite=True)

In [12]:
fname = added_path + f'EMOS_{cluster}_unsharp_1.img'
hdul = fits.open(fname)
data = hdul[0].data
header = hdul[0].header
    
hdul.close()

smooth_data = gaussian_filter(data, sigma=0.7)

hdu = fits.PrimaryHDU(smooth_data, header)
hdu.writeto(added_path + f'EMOS_{cluster}_unsharp_smooth.img', overwrite=True)